<a href="https://colab.research.google.com/github/MarcSaghir/testrepo/blob/main/EthicsNLPhw01_new1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import spacy
from tqdm import tqdm
nlp = spacy.load("en_core_web_sm")
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud


lgbt_news = pd.read_csv("lgbt_news_corpus.csv", encoding="ISO-8859-1")
background_news = pd.read_csv("background_news_corpus.csv", encoding="ISO-8859-1")
#print(background_news.columns)
seeds = pd.read_csv("seed_list.txt", names=["seed"])
#print(seeds)


In [ ]:
#task 3 begin

In [1]:
import json
import pandas as pd
import numpy as np
from tqdm import tqdm
import spacy
nlp = spacy.load("en_core_web_sm")

In [10]:
data = []
with open("snli_1.0_train.jsonl") as f:
  for line in f:
    #print(line)
    try:
      data.append(json.loads(line))
    except:
      print("ajo")
      print(line)
print(data[0].keys())

ajo
{"annotator_labels": ["neutral", "neutral", "neutral", "neutral", "neutral"], "captionID": "4689485382.jpg#0", "gold_label": "neutral", "pairID": "4689485382.jpg#0r1n", "sentence1": "An indian lady wearing orange is at the market in front of a bunch of fabric on a table talking to the lady sitting down.", "sentence1_binary_parse": "( ( ( An ( indian lady ) ) ( wearing orange ) ) ( ( ( is ( ( ( at ( the market ) ) ( in ( front ( of ( ( a bunch ) ( of fabric ) ) ) ) ) ) ( on ( a table ) ) ) ) ( talking ( to ( ( the lady ) ( sitting down ) ) ) ) ) . ) )", "sentence1_parse": "(ROOT (S (NP (NP (DT An) (NN indian) (NN lady)) (VP (VBG wearing) (ADJP (JJ orange)))) (VP (VBZ is) (ADVP (IN at) (NP (DT the) (NN market)) (PP (IN in) (NP (NP (NN front)) (PP (IN of) (NP (NP (DT a) (NN bunch)) (PP (IN of) (NP (NN fabric))))))) (PP (IN on) (NP (DT a) (NN table)))) (VP (VBG talking) (PP (TO to) (NP (NP (DT the) (NN lady)) (VP (VBG sitting) (PRT (RP down))))))) (. .)))", "sentence2": "A lady is at t

In [7]:
print(len(data))

75983


In [8]:
df = pd.DataFrame([])
sents = []
for d in tqdm(data):
    sents.append(d["sentence1"])
    sents.append(d["sentence2"])
df["sentences"] = sents

100%|██████████| 75983/75983 [00:00<00:00, 1350514.66it/s]


In [11]:
print(df.head())
print(len(df))

                                           sentences
0  A person on a horse jumps over a broken down a...
1  A person is training his horse for a competition.
2  A person on a horse jumps over a broken down a...
3      A person is at a diner, ordering an omelette.
4  A person on a horse jumps over a broken down a...
151966


In [12]:
unique_df = df.drop_duplicates().copy()

In [13]:
docs = unique_df["sentences"].tolist()

def token_filter(token):
    return not (token.is_punct | token.is_space | token.is_stop)

filtered_tokens = []
for doc in nlp.pipe(docs):
    tokens = [token.lower_ for token in doc if token_filter(token)]
    filtered_tokens.append(tokens)

unique_df['tokenized'] = filtered_tokens

In [14]:
print(unique_df.head())

                                           sentences  \
0  A person on a horse jumps over a broken down a...   
1  A person is training his horse for a competition.   
3      A person is at a diner, ordering an omelette.   
5                  A person is outdoors, on a horse.   
6              Children smiling and waving at camera   

                                  tokenized  
0  [person, horse, jumps, broken, airplane]  
1    [person, training, horse, competition]  
3       [person, diner, ordering, omelette]  
5                 [person, outdoors, horse]  
6       [children, smiling, waving, camera]  


In [15]:
#word frequencies in corpus
word_frequencies = {}
for sen in unique_df["tokenized"].tolist():
    for s in sen:
        if s in word_frequencies:
            word_frequencies[s] += 1
        else:
            word_frequencies[s] = 1
            
#print(word_frequencies)

In [19]:
"""testing"""



def get_mutual_occurence(word1, word2, corpus):
    return len(corpus.loc[
                lambda test_df: test_df.tokenized.apply(
                    lambda l: word1 in l and word2 in l
                )
            ])
        

def calculate_pmi(word, corpus, frequencies, threshold=10):
    #iterate through words in corpus(frequencies) and search for
    pmi_ranking = []
        
    
    keys, values = zip(*frequencies.items())
    tups = list(zip(keys,values))
    #print(len(tups))
    tups = [t for t in tups if t[1] > threshold]
    #print(len(tups))
    t = list(zip(*tups))
    keys = list(t[0])
    values = list(t[1])
    
    values = np.array(values)
    
    mut_occs = np.array([get_mutual_occurence(word, f, corpus) for f in tqdm(keys)], dtype=float)
    
    pmis = np.divide(np.multiply(float(len(corpus)), mut_occs), np.multiply(float(frequencies[word]), values))
    
    ranking = np.log2(pmis)
    
    res = list(zip(keys, ranking))
    
    return res

"""testing ende"""

'testing ende'

In [20]:
data = pd.read_csv("identity_labels.txt", names=["labels"])

rankings = []
for label in data["labels"]:
  ranking = calculate_pmi(label, unique_df, word_frequencies)
  rankings.append((label, ranking))
  break

#ranking = calculate_pmi()

100%|██████████| 3197/3197 [02:28<00:00, 21.58it/s]
<ipython-input-19-a9824d81fc3a>:33: RuntimeWarning: divide by zero encountered in log2
  ranking = np.log2(pmis)


In [21]:
print(rankings)

[('woman', [('person', -3.1982114265401367), ('horse', -0.0974498307356798), ('jumps', -0.5588920173621839), ('broken', -1.3105043874927915), ('airplane', -1.060526134484444), ('training', -1.154853517032597), ('competition', -0.8616420105656817), ('diner', -1.4360352695766505), ('ordering', 0.41196163697829963), ('outdoors', -0.500502803454811), ('children', -0.9894465218713177), ('smiling', 0.6416670874165934), ('waving', -2.4360352695766507), ('camera', 0.7039274938497946), ('parents', -2.745890532163438), ('present', 0.8031523943027756), ('kids', -1.6129738727800171), ('frowning', 0.4264612066734146), ('boy', -2.072344650298027), ('jumping', -1.0300770642086687), ('skateboard', -2.226112200202419), ('middle', 0.5223856266719501), ('red', 0.436278455308196), ('bridge', 0.23847881612279798), ('skates', -2.1730008637428564), ('sidewalk', 0.7754688356170613), ('skateboarding', -1.6990696754104442), ('trick', -3.1116003190787134), ('wearing', 0.5095406946629494), ('safety', -0.938535610

In [ ]:
from multiprocessing import Pool
import multiprocessing
from functools import partial


def get_mutual_occurence(word1, word2, corpus):
    mut_occ = np.array([has_mutual_occurence(x, word1, word2) for x in corpus], dtype=bool)
        
    return mut_occ.sum()

def has_mutual_occurence(sentence, word1, word2):
    return word1 in sentence and word2 in sentence

def calculate_pmi(word, corpus, frequencies, threshold=10):
    #iterate through words in corpus(frequencies) and search for
    pmi_ranking = []
    
    keys, values = zip(*frequencies.items())
    tups = list(zip(keys,values))
    #print(len(tups))
    tups = [t for t in tups if t[1] > threshold]
    #print(len(tups))
    t = list(zip(*tups))
    keys = list(t[0])
    values = list(t[1])
    
    values = np.array(values)
    
    mut_occs = np.array([get_mutual_occurence(word, f, corpus) for f in tqdm(keys)], dtype=float)
    
    pmis = np.divide(np.multiply(float(len(corpus)), mut_occs), np.multiply(float(frequencies[word]), values))
    
    ranking = np.log2(pmis)
    
    res = list(zip(keys, ranking))
    
    return ranking
        

In [ ]:
data = pd.read_csv("identity_labels.txt", names=["labels"])

rankings = []
for label in data["labels"]:
    ranking = calculate_pmi(label, unique_df, word_frequencies)
    rankings.append((label, ranking))
    break

#ranking = calculate_pmi()







  0%|                                                                                         | 0/8781 [00:00<?, ?it/s]





  0%|                                                                               | 1/8781 [00:00<1:01:51,  2.37it/s]





  0%|                                                                                 | 2/8781 [00:00<57:39,  2.54it/s]





  0%|                                                                                 | 3/8781 [00:01<54:23,  2.69it/s]





  0%|                                                                                 | 4/8781 [00:01<51:01,  2.87it/s]





  0%|                                                                                 | 5/8781 [00:01<48:45,  3.00it/s]





  0%|                                                                                 | 6/8781 [00:01<47:40,  3.07it/s]





  0%|                                                                                 | 7/8781 [00:02<46:38,  3.14it/s]

  1%|▌                                                                               | 57/8781 [00:17<45:35,  3.19it/s]





  1%|▌                                                                               | 58/8781 [00:18<44:53,  3.24it/s]





  1%|▌                                                                               | 59/8781 [00:18<45:31,  3.19it/s]





  1%|▌                                                                               | 60/8781 [00:18<45:36,  3.19it/s]





  1%|▌                                                                               | 61/8781 [00:19<44:52,  3.24it/s]





  1%|▌                                                                               | 62/8781 [00:19<45:34,  3.19it/s]





  1%|▌                                                                               | 63/8781 [00:19<48:21,  3.01it/s]





  1%|▌                                                                               | 64/8781 [00:20<47:32,  3.06it/s]







  1%|█                                                                              | 114/8781 [00:35<45:23,  3.18it/s]





  1%|█                                                                              | 115/8781 [00:36<44:51,  3.22it/s]





  1%|█                                                                              | 116/8781 [00:36<44:29,  3.25it/s]





  1%|█                                                                              | 117/8781 [00:36<44:36,  3.24it/s]





  1%|█                                                                              | 118/8781 [00:37<44:32,  3.24it/s]





  1%|█                                                                              | 119/8781 [00:37<44:18,  3.26it/s]





  1%|█                                                                              | 120/8781 [00:37<44:07,  3.27it/s]





  1%|█                                                                              | 121/8781 [00:38<44:07,  3.27it/s]







  2%|█▌                                                                             | 171/8781 [00:53<42:56,  3.34it/s]





  2%|█▌                                                                             | 172/8781 [00:53<42:55,  3.34it/s]





  2%|█▌                                                                             | 173/8781 [00:54<42:54,  3.34it/s]





  2%|█▌                                                                             | 174/8781 [00:54<43:22,  3.31it/s]





  2%|█▌                                                                             | 175/8781 [00:54<44:10,  3.25it/s]





  2%|█▌                                                                             | 176/8781 [00:55<45:20,  3.16it/s]





  2%|█▌                                                                             | 177/8781 [00:55<44:37,  3.21it/s]





  2%|█▌                                                                             | 178/8781 [00:55<45:12,  3.17it/s]







  3%|██                                                                             | 228/8781 [01:11<43:44,  3.26it/s]





  3%|██                                                                             | 229/8781 [01:11<43:21,  3.29it/s]





  3%|██                                                                             | 230/8781 [01:11<43:12,  3.30it/s]





  3%|██                                                                             | 231/8781 [01:11<43:17,  3.29it/s]





  3%|██                                                                             | 232/8781 [01:12<43:26,  3.28it/s]





  3%|██                                                                             | 233/8781 [01:12<43:19,  3.29it/s]





  3%|██                                                                             | 234/8781 [01:12<43:35,  3.27it/s]





  3%|██                                                                             | 235/8781 [01:13<43:12,  3.30it/s]







  3%|██▌                                                                            | 285/8781 [01:28<42:57,  3.30it/s]





  3%|██▌                                                                            | 286/8781 [01:28<43:12,  3.28it/s]





  3%|██▌                                                                            | 287/8781 [01:29<43:46,  3.23it/s]





  3%|██▌                                                                            | 288/8781 [01:29<43:10,  3.28it/s]





  3%|██▌                                                                            | 289/8781 [01:29<42:39,  3.32it/s]





  3%|██▌                                                                            | 290/8781 [01:29<43:01,  3.29it/s]





  3%|██▌                                                                            | 291/8781 [01:30<42:47,  3.31it/s]





  3%|██▋                                                                            | 292/8781 [01:30<42:58,  3.29it/s]







  4%|███                                                                            | 342/8781 [01:45<42:44,  3.29it/s]





  4%|███                                                                            | 343/8781 [01:46<42:36,  3.30it/s]





  4%|███                                                                            | 344/8781 [01:46<43:01,  3.27it/s]





  4%|███                                                                            | 345/8781 [01:46<43:03,  3.27it/s]





  4%|███                                                                            | 346/8781 [01:47<43:02,  3.27it/s]





  4%|███                                                                            | 347/8781 [01:47<42:42,  3.29it/s]





  4%|███▏                                                                           | 348/8781 [01:47<42:35,  3.30it/s]





  4%|███▏                                                                           | 349/8781 [01:47<42:35,  3.30it/s]







KeyboardInterrupt: 

In [ ]:
#task 3 end

In [ ]:
analytics = {}

#cycle through seeds
for s in seeds["seed"]:
    print(s)
    dicto = {}
    #cycle through both corpus
    for b in tqdm(lgbt_news["Text"][:30]):
        #print(b)
        try:
            doc = nlp(b)
        except:
            print("continued")
            continue
        for i in range(len(doc)):
            #if s == "gay":
            #    print(doc[i].lower_)
            if doc[i].lower_ == s.lower():
                lower_bound = i
                upper_bound = i
                
                context = []
                #window size = 3
                #3 previous words
                while len(context) != 3:
                    lower_bound -= 1
                    if lower_bound < 0:
                        break
                    elif not doc[lower_bound].is_alpha:
                        continue
                    else:
                        context.insert(0, doc[lower_bound].lower_)
                        
                #3 words after seed
                l = len(context)
                while len(context) != l + 3:
                    upper_bound += 1
                    if upper_bound >= len(doc):
                        break
                    elif not doc[upper_bound].is_alpha:
                        continue
                    else:
                        context.append(doc[upper_bound].lower_)
                    
                
                #analyse window
                #context = list(doc[i-3:i]) + list(doc[i+1:i+4])
                #print(context)
                
                
                for c in context:
                    if c not in dicto:
                        dicto[c] = 1
                    else:
                        dicto[c] += 1
    analytics[s] = dicto
                    

agender


100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 37.55it/s]


androgyny


100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 40.47it/s]


androgynyous


...

100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 44.07it/s]


homosexual


100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 47.22it/s]


homosexuals


100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 48.44it/s]


In [ ]:
print(analytics)

{'agender': {}, 'androgyny': {}, 'androgynyous': {}, 'bisexual': {}, 'bigender': {}, 'butch': {}, 'cisgender': {}, 'gay': {'hospital': 1, 'rules': 1, 'for': 2, 'front': 1, 'page': 1, 'april': 1, 'the': 8, 'community': 1, 'segment': 1, 'black': 1, 'members': 1, 'who': 2, 'are': 3, 'she': 2, 'said': 1, 'adding': 1, 'a': 4, 'magazine': 1, 'men': 4, 'and': 12, 'lesbians': 2, 'spectrum': 1, 'of': 4, 'political': 1, 'opinion': 1, 'go': 1, 'to': 4, 'web': 1, 'site': 1, 'means': 1, 'is': 3, 'that': 5, 'scouts': 1, 'leaders': 1, 'fears': 1, 'by': 1, 'priests': 1, 'seminarians': 1, 'document': 1, 'will': 1, 'bar': 1, 'even': 1, 'those': 1, 'director': 1, 'health': 1, 'crisis': 1, 'state': 1, 'allow': 1, 'human': 1, 'rights': 1, 'message': 1, 'but': 2, 'many': 1, 'people': 2, 'in': 3, 'their': 1, 'chad': 1, 'his': 1, 'town': 1, 'or': 2, 'coming': 1, 'our': 1, 'son': 1, 'should': 1, 'we': 1, 'gentleman': 1, 'yet': 1, 'another': 1, 'couple': 1, 'providing': 1, 'an': 1, 'lesbian': 3, 'students': 1, 

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud
 
#assume that this is the dictionary, feel free to change it
word_could_dict = {'Git':100, 'GitHub':100, 'push':50, 'pull':10, 'commit':80, 
                    'add':30, 'diff':10, 'mv':5, 'log':8, 'branch':30, 'checkout':25}
 
wordcloud = WordCloud(font_path=r'.\SourceHanSerifK-Light.otf', width = 1000, height = 500).generate_from_frequencies(word_could_dict)
 
plt.figure(figsize=(15,8))
plt.imshow(wordcloud)

AttributeError: 'TransposedFont' object has no attribute 'getbbox'

In [ ]:
import os

from os import path
from wordcloud import WordCloud

word_could_dict = {'Git':100, 'GitHub':100, 'push':50, 'pull':10, 'commit':80, 
                    'add':30, 'diff':10, 'mv':5, 'log':8, 'branch':30, 'checkout':25}

# Generate a word cloud image
wordcloud = WordCloud().generate_from_frequencies(word_could_dict)

# Display the generated image:
# the matplotlib way:
import matplotlib.pyplot as plt
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")

# lower max_font_size
wordcloud = WordCloud(max_font_size=40).generate_from_frequencies(word_could_dict)
plt.figure()
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

AttributeError: 'TransposedFont' object has no attribute 'getbbox'